In [53]:
import pandas as pd
import numpy as np
import re, urllib.parse, matplotlib

In [54]:
df_main = pd.read_csv("properties_data.csv")
df = df_main[["id","location","Zip","Type","Subtype",
    "Price","Transaction Type","Bedrooms","Living area","Kitchen type",
    "Furnished","How many fireplaces?","Terrace","Terrace surface",
    "Garden","Garden surface","Surface of the plot","Number of frontages",
    "Swimming pool","Building condition","Primary energy consumption","CO₂ emission"]]
df = df.set_index("id")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_94458/392842248.py:1: DtypeWarning: Columns (41,42,43,44,90,91,92,93,113,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df_main = pd.read_csv("properties_data.csv")


In [55]:
df = df.drop(df[df["Type"]=="house group"].index)
df = df.drop(df[df["Type"]=="apartment group"].index)

In [56]:
def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan)
    return column

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace surface'] = clean_and_convert(df['Terrace surface'])
df['Garden surface'] = clean_and_convert(df['Garden surface'])
df['Surface of the plot'] = clean_and_convert(df['Surface of the plot'])
df['Primary energy consumption'] = clean_and_convert(df['Primary energy consumption'])
df['CO₂ emission'] = clean_and_convert(df['CO₂ emission'])

df['location'] = df['location'].apply(urllib.parse.unquote)

In [57]:
df["Garden"] = np.where((df["Garden"] == "Yes") & (df["Garden surface"] == ""), 1, df["Garden"])
df["Garden"] = np.where((df["Garden"] != "Yes") & (df["Garden surface"] != ""), 1, df["Garden"])
df["Garden"] = df["Garden"].replace("",np.nan).fillna(0)

df["Terrace"] = np.where((df["Terrace"] == "Yes") & (df["Terrace surface"] == ""), 1, df["Terrace"])
df["Terrace"] = np.where((df["Terrace"] != "Yes") & (df["Terrace surface"] != ""), 1, df["Terrace"])
df["Terrace"] = df["Terrace"].replace("",np.nan).fillna(0)

In [58]:
def nan_replacement(column):
    column = column.replace("Yes",1)
    column = column.replace("No",0)
    column = column.replace('', np.nan).fillna(0)
    return column

df['Furnished'] = nan_replacement(df['Furnished'])
df['Swimming pool'] = nan_replacement(df['Swimming pool'])
df['How many fireplaces?'] = nan_replacement(df['How many fireplaces?'])

In [59]:
df["Price"] = df["Price"].astype(int)
df = df.astype({"Price":"int",
                "Bedrooms":"float",
                "Living area":"float",
                "Terrace surface":"float",
                "Garden surface":"float","Surface of the plot":"float",
                "Number of frontages":"float","Primary energy consumption":"float",
                "CO₂ emission":"float"})

In [60]:
# Mapping dictionary for replacing values in the "kitchen" column
kitchen_mapping = {
    np.nan: -1,
    'Not installed': 0,
    'Installed': 1,
    'Semi equipped': 2,
    'Hyper equipped': 3,
    'USA uninstalled' :0,
    'USA installed': 1,
    'USA semi equipped': 2,
    'USA hyper equipped' :3
}
# Replace values in the "Kitchen type" column with corresponding numbers and create a new column called "Kitchen values"
df['Kitchen values'] = df['Kitchen type'].map(kitchen_mapping).fillna(df['Kitchen type']).astype(int)

In [61]:
building_cond_mapping = {
    np.nan: -1,
    'To restore': 0,
    'To be done up': 0,
    'Just renovated': 2,
    'To renovate': 1,
    'Good': 2,
    'As new' :3
}

df['Building Cond. values'] = df['Building condition'].map(building_cond_mapping).fillna(df['Building condition']).astype(int)

In [62]:
df = df.drop(df[df["Living area"].isna()].index)

In [63]:
df.isna().sum()

location                          0
Zip                               0
Type                              0
Subtype                           0
Price                             0
Transaction Type                  0
Bedrooms                        331
Living area                       0
Kitchen type                   3933
Furnished                         0
How many fireplaces?              0
Terrace                           0
Terrace surface                9555
Garden                            0
Garden surface                13425
Surface of the plot            8985
Number of frontages            4202
Swimming pool                     0
Building condition             2843
Primary energy consumption     2557
CO₂ emission                  12477
Kitchen values                    0
Building Cond. values             0
dtype: int64

In [ ]:
df.to_csv("clean_data.csv")